In [1]:
import pandas as pd
import os

In [2]:
master      = pd.read_pickle(os.path.join("..", "master.pickle"))
scoring     = pd.read_pickle(os.path.join("..", "scoring.pickle"))
team_splits = pd.read_pickle(os.path.join("..", "team_splits.pickle"))

In [3]:
scoring = scoring.reset_index()
scoring = scoring.drop(columns='_merge')

In [4]:
# multi level index
mi = scoring.set_index(['playerID', 'year'])
mi.head()

index firstName    lastName pos tmID    GP    G     A   Pts  \
playerID  year                                                                  
aaltoan01 1998.0      4     Antti       Aalto   C  ANA  73.0  3.0   5.0   8.0   
          1999.0      5     Antti       Aalto   C  ANA  63.0  7.0  11.0  18.0   
          2000.0      6     Antti       Aalto   C  ANA  12.0  1.0   1.0   2.0   
abdelju01 2007.0      7    Justin  Abdelkader   L  DET   2.0  0.0   0.0   0.0   
          2008.0      8    Justin  Abdelkader   L  DET   2.0  0.0   0.0   0.0   

                    SOG  
playerID  year           
aaltoan01 1998.0   61.0  
          1999.0  102.0  
          2000.0   18.0  
abdelju01 2007.0    6.0  
          2008.0    2.0

In [5]:
#mi.index
mi = mi.sort_index()

In [6]:
mi.index.nlevels  # same as : len(mi.index.levels)

2

In [13]:
mi.index.levels[0]

Index(['aaltoan01', 'abdelju01', 'abidra01', 'abrahth01', 'actonke01',
       'adamlu01', 'adamru01', 'adamsbr01', 'adamscr01', 'adamsgr01',
       ...
       'zolniha01', 'zombori01', 'zubaran01', 'zubovil01', 'zubovse01',
       'zubruda01', 'zuccama01', 'zuckeja01', 'zukemi01', 'zyuzian01'],
      dtype='object', name='playerID', length=4627)

In [14]:
mi.index.levels[1]

Float64Index([1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0, 1986.0, 1987.0,
              1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0, 1995.0,
              1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0,
              2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0],
             dtype='float64', name='year')

In [16]:
mi.groupby(level="year")['G'].max().head()

year
1980.0    68.0
1981.0    92.0
1982.0    71.0
1983.0    87.0
1984.0    73.0
Name: G, dtype: float64

In [18]:
mi.groupby(level='year')['G'].idxmax().head()

year
1980.0    (bossymi01, 1980.0)
1981.0    (gretzwa01, 1981.0)
1982.0    (gretzwa01, 1982.0)
1983.0    (gretzwa01, 1983.0)
1984.0    (gretzwa01, 1984.0)
Name: G, dtype: object

In [19]:
mi.loc[mi.groupby(level="year")["G"].idxmax()].head()

index firstName lastName pos tmID    GP     G      A    Pts  \
playerID  year                                                                  
bossymi01 1980.0   2144      Mike    Bossy   R  NYI  79.0  68.0   51.0  119.0   
gretzwa01 1981.0   9042     Wayne  Gretzky   C  EDM  80.0  92.0  120.0  212.0   
          1982.0   9043     Wayne  Gretzky   C  EDM  80.0  71.0  125.0  196.0   
          1983.0   9044     Wayne  Gretzky   C  EDM  74.0  87.0  118.0  205.0   
          1984.0   9045     Wayne  Gretzky   C  EDM  80.0  73.0  135.0  208.0   

                    SOG  
playerID  year           
bossymi01 1980.0  315.0  
gretzwa01 1981.0  369.0  
          1982.0  348.0  
          1983.0  324.0  
          1984.0  358.0

In [20]:
mi.loc[('gretzwa01', 1982), :]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


,,index,firstName,lastName,pos,tmID,GP,G,A,Pts,SOG
playerID,year,,,,,,,,,,
gretzwa01,1982.0,9043,Wayne,Gretzky,C,EDM,80.0,71.0,125.0,196.0,348.0


In [23]:
idx = pd.IndexSlice
mi2 = mi.sort_index()

In [26]:
mi.index.is_lexsorted()

True

In [27]:
sliced = mi.loc[idx['aaltoan01':'adamscr01', 1997:2004:2], :]
sliced.head()

index firstName lastName pos tmID    GP    G     A   Pts  \
playerID  year                                                               
aaltoan01 1999.0      5     Antti    Aalto   C  ANA  63.0  7.0  11.0  18.0   
abidra01  2003.0     14     Ramzi     Abid   L  PIT  16.0  3.0   2.0   5.0   
adamsbr01 1999.0     39     Bryan    Adams   L  ATL   2.0  0.0   0.0   0.0   
adamscr01 2001.0     42     Craig    Adams   R  CAR  33.0  0.0   1.0   1.0   
          2003.0     44     Craig    Adams   R  CAR  80.0  7.0  10.0  17.0   

                    SOG  
playerID  year           
aaltoan01 1999.0  102.0  
abidra01  2003.0   35.0  
adamsbr01 1999.0    1.0  
adamscr01 2001.0   17.0  
          2003.0  110.0

In [28]:
import numpy as np

In [32]:
def get_many_locs(df, slices):
    arr = np.empty(0, dtype="int")
    for s in slices:
        locs = df.index.get_locs((s))  # input should be a tuple
        arr = np.concatenate((arr, locs))
    return arr

In [33]:
locs = get_many_locs(mi, (idx['aaltoan01':'adamscr01', 1997:2000],
                          idx['aaltoan01':'adamscr01', 2004:2006])
                    )
locs

array([ 0,  1,  2, 34, 35, 36, 11, 12, 40, 41])

In [35]:
sliced = mi.iloc[locs, :]
sliced.head(10)

index firstName lastName pos tmID    GP     G     A   Pts  \
playerID  year                                                                
aaltoan01 1998.0      4     Antti    Aalto   C  ANA  73.0   3.0   5.0   8.0   
          1999.0      5     Antti    Aalto   C  ANA  63.0   7.0  11.0  18.0   
          2000.0      6     Antti    Aalto   C  ANA  12.0   1.0   1.0   2.0   
adamsbr01 1999.0     39     Bryan    Adams   L  ATL   2.0   0.0   0.0   0.0   
          2000.0     40     Bryan    Adams   L  ATL   9.0   0.0   1.0   1.0   
adamscr01 2000.0     41     Craig    Adams   R  CAR  44.0   1.0   0.0   1.0   
abidra01  2005.0     15     Ramzi     Abid   L  ATL   6.0   0.0   2.0   2.0   
          2006.0     16     Ramzi     Abid   L  NAS  13.0   1.0   4.0   5.0   
adamscr01 2005.0     45     Craig    Adams   R  CAR  67.0  10.0  11.0  21.0   
          2006.0     46     Craig    Adams   R  CAR  82.0   7.0   7.0  14.0   

                    SOG  
playerID  year           
aaltoan01 1998.0   61.0  
          1999.0  102.0  
          2000.0   18.0  
adamsbr01 1999.0    1.0  
          2000.0    3.0  
adamscr01 2000.0   15.0  
abidra01  2005.0    6.0  
          2006.0   12.0  
adamscr01 2005.0   68.0  
          2006.0   71.0

In [7]:
mi.to_pickle(os.path.join("..", "scoring_mi.pickle"))